In [216]:
import pandas as pd
import numpy as np

Here i defined the column names and read the Dta 

In [219]:
colnames = ['buying','maint','doors','persons','lug_boot','safety','class']
Data =pd.read_csv("car.data", delimiter =",",names=colnames, header=None)
y = Data['safety']
X = Data.drop(columns=['safety'], axis=1)
Data.head()
#print(target)





,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


Splitting the data, training, validation , test<br>
strasfied sampling

In [225]:
  Data_high = Data[Data['safety'] == 'high']
  Data_low = Data[Data['safety'] == 'low']
  Data_med = Data[Data['safety'] == 'med']
  data_train  = pd.concat([Data_high[0: int(0.7*Data_high.shape[0])],
                          Data_low[0: int(0.7*Data_low.shape[0])],
                          Data_med [0: int(0.7*Data_med.shape[0])]       ], 
                          ignore_index=True)
   
  data_val  = pd.concat([Data_high[ int(0.7*Data_high.shape[0]):int(0.85*Data_high.shape[0])],
                          Data_low[int(0.7*Data_low.shape[0]):int(0.85*Data_low.shape[0]) ],
                          Data_med [int(0.7*Data_med.shape[0]): int(0.85*Data_low.shape[0])]       ], 
                          ignore_index=True)  

  data_test  = pd.concat([Data_high[int(0.85*Data_high.shape[0]):],
                          Data_low[int(0.85*Data_low.shape[0]) : ],
                          Data_med [int(0.85*Data_low.shape[0]): ]       ], 
                          ignore_index=True)



<b>Defing the class Tree_node</b> <br>
every node has a child , class_v which represents the class it belongs to, depth which represents the level 
of the node is in , and a boolean representing whether it is a leaf or a not and a name of the node
which represents the feature that it is splitted on

In [226]:
class Tree_node:
    def __init__(self,
                 split,
                 child,
                 class_v,
                 depth ):
        self.split = split
        self.child = child
        self.class_v = class_v
        self.depth = depth
        self.isLeaf = False
        self.Name =  '' 
        


<b>Explanantion of checking_stopping equation</b><br>

this function we use to check the stopping criteria<br>

1. we give it the maxdepth we want to stop when reaching it, when we use reach this we rerurn True 





In [234]:
def check_stopping (T,max_depth):
    if (T.depth > max_depth):
        return True
    else: 
       return False

<b>Explanation of the equation entropy<b><br>
1. we use this equation when calcuating info_gain <br>
2. it takes the unique number of classes<br>
3. for every class it calculates the entropy using the equation given in the slides

    
    

In [242]:
def entropy(data):
    # to get the unique classes
    classes,counts = np.unique(data, return_counts = True)
    info = dict(zip(classes,counts))
    total_num = len(data)
    # 
    entropy = 0
    for class_v in classes:
        entropy = entropy - (info[class_v]/total_num)*np.log2(info[class_v]/total_num)
    return entropy       

<b>Explanantion of the quality_Info_gain function<b><br>
1. It takes as input the Data and the target and the split_att_name<br>
2. first of all we calculate the entropy of the whole target<br>
3. Then get the number and name of each classes and then calculate the entropy after sliding using the equatios
 from the slide<br>
4. info_gain is the entropy_before_splitting - entropy_after_splitting      




In [240]:
def quality_info_gain(x,y,split_att_name ,target = 'safety'):
    #first calculate total entropy of the dataset
    entropy_data = entropy(y)
    #calculate entropy of every possible value 
    classes,counts  = np.unique(x[split_att_name],return_counts=True)
    info =  dict(zip(classes,counts))
    
    for class_v in classes:
        entropy_splitting = 0 
        #entropy_x = entropy(x[x[split_att_name] == class_v ])
        #print(entropy_x)                    
        entropy_splitting =entropy_splitting +(info[class_v]/sum(info.values())*entropy(x[x[split_att_name] == class_v ] ))
    # use the weight of each one to claculate the 
    #print(split_att_name ,info)
    gain = entropy_data - entropy_splitting 
    return gain

In [ ]:
def MCR():
    

In [86]:
def MCR(middle,left,right,left_t,right_t,middle_t):
    print('left is' , left)
    print ('right is', right)
    for row in left.iterrows():
        if (row['safety'] != left_t):
            error = error +1
    for row in middle.iterrows() :
        if (row['safety'] != right_t):
            error = error +1
    for roe in right.iterrows() :        
         if (row['safety'] != ):
            error = error +1
    MCR = error/(len(left)+len(right))
    

<b> Explanation of the best_split_function </b><br>
1.we loop over all the possible columns<br>
2. And for every column we go calculate the quality_info_gain defined above
3. we return the column which will produce the best gain 
    

In [238]:
def best_split(x,y,Q_critera = 'info_gain'):
     features = x.columns
     q_values = [0]* len(features)
     if Q_critera == 'info_gain':
            for i, column in enumerate(x.columns) :
                 #print(column)
                 #print("this is the return",quality_info_gain(x,y,column))
                 q_values[i]=(quality_info_gain(x,y,column))  #Return the information gain values for the features in the dataset  
            #print(q_values) 
            best_feature = features[np.argmax(q_values)] 
    
     #print(best_feature)
     return best_feature    
         
    

<b>Explanantion of expanding the TREE</b><br>
1. it takes a node to expand it along with the data, target and the depth<br>
2. we call stopping criterion to check whether or not we have reached the max depth <br>
3. if we reached we make it a leaf and get the majority class<br>
4. if not we call best_split to know which column to split and we increase the depth  <br>
5. for every possile vaue in s , we get a subset based on that value<br>
6. we define new node, T_prime<br>
7. now we equate a child of node T to be T_prime
8. we give the node T a name which is equal to the best_split
6. then we call a recursive call the expand_decision_tree on the child_t_prime

In [229]:
def expand_decision_tree(T, X,Y ,Depth):
    if check_stopping(T,5):
        classes, counts = np.unique(Y, return_counts=True)
        majority_class = classes[np.argmax(counts)]
        T.isLeaf = True
        T.class_v = majority_class
        return
    s = best_split(X,Y)
    Depth = Depth + 1
    for i,z in enumerate(np.unique(X[s])):  
            value = z  
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets  
            sub_data = X[X[s] == value]   
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!  
            T_prime = Tree_node(split =['empty']*5, child =['empty']*5, class_v ='',depth = Depth)
            T.child[i] = T_prime
            T.Name = best_split(X,Y)
            #attrs = vars(T)
            #print(attrs)
            expand_decision_tree(T_prime,X[X[s] == z],Y, Depth)
              
              
    
 

In [261]:
def print_Tree (T,count =0):
        print(T.depth, T.child,T.Name)
        if(T.isLeaf == True):
            print("This is the class",T.class_v)
        else: 
            print("The name of the node is",T.Name )
            for i,child in enumerate(T.child):
                if(T.child[i].Name == 'empty'):
                    continue
                count = count +1
                print("the Depth of the node is ", T.depth)
                print("The name of the cild_node",T.child[i].Name)
                print("-------------------------------")
                
                print_Tree(T.child[i], count )
            

In [262]:

y = data_train['safety'] 
x = data_train.drop(['safety'], axis=1)

def learn_decision_tree(x,y):
    T = Tree_node(split =['empty']*5, child =['empty']*5, class_v ='',depth =0)
    #attrs = vars(T)
    #print(attrs)
    expand_decision_tree(T,x, y,Depth=1)
    return T
        
T = learn_decision_tree(x,y)
print_Tree (T)


    
    

0 [<__main__.Tree_node object at 0x1a1c363750>, <__main__.Tree_node object at 0x1a1e0b4310>, <__main__.Tree_node object at 0x1a1e099190>, <__main__.Tree_node object at 0x1a1e0c7210>, 'empty'] class
The name of the node is class
the Depth of the node is  0
The name of the cild_node maint
-------------------------------
2 [<__main__.Tree_node object at 0x1a1e0cc550>, <__main__.Tree_node object at 0x1a1e0a5950>, <__main__.Tree_node object at 0x1a1e08e8d0>, <__main__.Tree_node object at 0x1a1c592890>, 'empty'] maint
The name of the node is maint
the Depth of the node is  2
The name of the cild_node lug_boot
-------------------------------
3 [<__main__.Tree_node object at 0x1a1e0cc910>, <__main__.Tree_node object at 0x1a1e0cca90>, <__main__.Tree_node object at 0x1a1e09fb90>, 'empty', 'empty'] lug_boot
The name of the node is lug_boot
the Depth of the node is  3
The name of the cild_node doors
-------------------------------
4 [<__main__.Tree_node object at 0x1a1e0cccd0>, <__main__.Tree_node

AttributeError: 'str' object has no attribute 'Name'

As we can see in the print after the learn_descion_tree algorithim i'm not printing correctly
but some things ca be identfied . <br>

1.The name of the node we are splitting on<br>
2.The list of child nodes as objects<br>
3.The depth of every node <br>
4.we stop at depth 5, which i defined as thw stopping criteria<br>
5.The leaf is printed its class <br>
6. whenever the sentence "This is the class appears" it appears with it an empty_list because it is aleaf therefore the childs are empty<br>
7. I think the printing only shows splitting of 1 branch only and this is the problem, u can see that the child node in a depth n-1 is always the node in n and the other are printed as objects so i think this is where the problem of printing<br>
8. the number that appears beside the list is the column number that is used in spltting <b> Not the Depth</b>



<b>Exercise 2<b>

In [4]:
from sklearn.datasets import make_moons
import numpy as np

In [5]:
def sig(x):
    return  1/(1 + np.exp(-x))

In the below cell we generate the x and the y 

and we divide the value with strasfied_sampling to x,y and x_val,y_val and x_test,y_test

In [266]:

X = np.zeros((1000,2))
y = np.zeros((1000))



for i in range (0,10):
    start = i*100
    end = (i*100)+100
    X[start:end,:] = make_moons(n_samples=100, noise=0.01*i)[0]
    y[start:end] = make_moons(n_samples=100, noise=0.01*i)[1]
 
'''
# test with the example in the slides #
X= np.array([[1,2],[2,1],[3,2],[1,3],[2,2],[3,3]])
y= np.array([0,0,0,1,1,1])
'''
indices_zero = np.where(y==0)
indices_one  = np.where( y==1)

#print(indices_zero)
#print(indices_one)
x_zero = X[indices_zero]
y_zero = y[indices_zero]
x_one  = X[indices_one]
y_one =  y[indices_one]



seventy_per_zero = int(0.7*len(x_zero))
seventy_per_zero_y = int(0.7*len(y_zero))
eightfive_per_zero = int(0.85*len(x_zero))
seventy_per_one = int(0.7*len(x_one))
eightfive_per_one = int(0.85*len(x_one))

X = np.concatenate((x_zero[:seventy_per_zero],x_one[:seventy_per_one]), axis =0)
y = np.concatenate((y_zero[:seventy_per_zero],y_one[:350]), axis =0)

x_val = np.concatenate((x_zero[seventy_per_zero:eightfive_per_zero],x_one[seventy_per_one:eightfive_per_one ]), axis =0)
y_val = np.concatenate((y_zero[seventy_per_zero:eightfive_per_zero],y_one[seventy_per_one:eightfive_per_one]), axis =0)
x_test = np.concatenate((x_zero[eightfive_per_zero:],x_one[eightfive_per_one:]), axis =0)                       
y_test = np.concatenate((y_zero[eightfive_per_zero:],y_one[eightfive_per_one:]), axis =0)


print ( "length of x_train, length of y_train",len(X), ',', len(Y))
print ("length of x_val, y_val",len(x_val),',', len(y_val))
print("length of x_test,y_test",len(x_test),',',len(y_test))


   

length of x_train, length of y_train 700 , 700
length of x_val, y_val 150 , 150
length of x_test,y_test 150 , 150


<b>Explanantion of get_split_value<b><br>

1.we get the unique values in a column and sort them <br>
2.we fill the split values_array with the avg value between evry 2 values<br>
3.we return the array  


In [50]:

def get_split_value(column):
     unique = set(column)
     unique_sorted = sorted(unique)
     split_values = [0 ]* (len(unique_sorted)-1)
     #print (len(split_values))   
     for i, val in enumerate(unique_sorted):
            if i != (len(unique_sorted)-1):
                split_value = (unique_sorted[i]+unique_sorted[i+1])/2
                split_values[i] = split_value
            
            
     return split_values

<b> Explanantion of the gain measurment <b>

1. the implmentation of the gain is based on slide 19 in the provided lecture<br>

2. the function takes as an input the index of  left branch, index right branch, gradients, hessians and the regularization factor<br>

3. we calculate the sum of the gradients in the left, sum of the gradients in the right, sum of the gradient of the parent
which is left+right<br>

4. we do the same for the hessians<br>

5. we square the gradients<br>

6. we return the gain which is 0.5(square_G_left / (lamb + sum_of_hessian_left )+ square_G_right / (lamb + sum_of_hessian_right) - square_parent_gadient/(lamb +parent_hessian)) - reg_factor 

In [182]:
def gain_measurment(left,right,G,H,lamb=1):
    sum_of_grad_left = 0
    sum_of_grad_right = 0
    sum_of_hessian_left = 0
    sum_of_hessian_right = 0
    for index in left:
        sum_of_grad_left  = sum_of_grad_left + G[index]
        sum_of_hessian_left = sum_of_hessian_left +H[index] 
    for index in right:    
        sum_of_grad_right = sum_of_grad_right + G[index]
        sum_of_hessian_right = sum_of_hessian_right +H[index]
        
    parent_gradient = sum_of_grad_left + sum_of_grad_right
    parent_hessian  = sum_of_hessian_left + sum_of_hessian_right 
    square_parent_gadient = parent_gradient**2
    square_left = sum_of_grad_left**2
    square_right = sum_of_grad_right**2
    
    left_variable = square_left / (lamb + sum_of_hessian_left)
    right_variable = square_right / (lamb + sum_of_hessian_right)
    parent_variable = square_parent_gadient/(lamb +parent_hessian) 
    gain = 0.5*(left_variable + right_variable - parent_variable ) - lamb
    
    return gain

<b> Explanantion of split function<b> <br>
1. the function takes the data, the  best column to split on and the best value to split on <br>
2. if the data is less than or equal to the value put it in the left<br>
3. if the data is greater than put it inn the right<br>
4. return the left and the right<br>

In [52]:
def split(X,column, value): 
    left = np.where(X[:,column] <= value)
    right = np.where(X[:,column] > value)
    left = np.unique(left[0])
    right = np.unique(right[0])
    return left,right    

<b>Explanation function getting_best_split_paramters</b> <br>
We use this function to get the best_feature and best value to split on. <br>
1.We loop over all the columns
(features) and for every feature we try every split value which is the result of calling
<b>function get_split_value</b> on<br>
2.for every left and right we call <b>function gain_measurment</b> and we finally take the left and the right
with the best_gain<br>
3. we return which column is the best to split on as well as the value to split_on and the best_gain

In [207]:
def getting_best_split_paramters(X,G,H):
        best_gain = -5
        best_column = ''
        best_value = 0
        columns = X.shape[1]
        for i in range(columns):
          split_values = get_split_value(X[:,i])
          for split_value in split_values :
              left,right = split(X,i,split_value )  
              gain = gain_measurment(left,right,G,H,1) 
              if (gain > best_gain ):
                  best_gain = gain
                  best_column = i
                  best_value = split_value 
        return best_column,best_value,best_gain            

<b> Explanantion of the gradient_boositng  function </b><br>
1.it take as input the data and the target and the tree_num<br>
2.first of all we generate y_hat and for the first_tree they are zeroes then for the next trees they are added in the varaible called additive variable <br>
3. additive variaible which will be added to the y_hat in the next trees is initalized to zero<br>
4. Then i generate the gradients and the hessians which we will need in the gain calculation using the equations in the slides given<br>
6. we call best_slide_paramters function to get the best_column and best value to split on<br>
7. Then we call the split function to do the actual splitting using info from point 6, indices of left and right
are returned<br>  
8. we add 1 to the depth because we went one level down<br>
9. we loop over the max_depth and because we max_depth =2 we know that only one more splitting will be done,
if the max_depth is more than 2 then we should defined a data structure to save data and then get classes, but we know only one more
splitting done so which means we will have 4 leaves<br> 
10. the leaves left_l is left_left and left_r is left_right and right_l is right_left and right_r is right_right<br>
11. we get the indices of those leaves by spliing the first level and right<br>

12. After splitting, we know need to get the majority class of each_leaf which and save it in class_left_l 
and so on<br> 
13. now we calculate the error in the left branch and right branch as a whole by comparing the the majotiry class
in every leaf and their true label<br>
14.we modify the variable the additive variabel so it would be ready for the next tree<br>
15. we silice the additive variable and ecery variable is equated by the value of ther -gradients/hessians+lamb<br>
16. we add the error in the dictionary<br>  







In [263]:
def gradient_boosting(X,Y, tree_num = 1, depth = 0, max_depth = 2,lamb =1):
    error = {}
    #print(
    y_hat = np.zeros(Y.shape[0])
    # starting off the additive_var = 0 and its value is updated with every new tree                         
    additive_var = np.zeros(y_hat.shape[0]) 
                             
    for tree_val in range(tree_num):
                             
     # updating the gradients and the hessians based on which tree we are in                         
        y_hat = y_hat + additive_var
        #print(y_hat)
        sig_y_hat = sig(y_hat)
        G = sig_y_hat - Y
        H = sig_y_hat *( 1 - sig_y_hat)
                             
        
          
        
        best_column,best_value,best_gain = getting_best_split_paramters(X,G,H)
        print(best_column,best_value,best_gain)
        #best_value  = getting_best_split_paramters(X)[1]
        #best_gain =  getting_best_split_paramters(X)[2]
      # splitting after i got the best parmters from the function getting_best_split_paramters
        left,right = split(X,best_column,best_value)
        depth = depth+1
     # i will keep splitting until i reached the max_depth   
        for d in range (depth, max_depth):
             print("this is the depthh",d)
             best_column_L,best_value_L,best_gain= getting_best_split_paramters(X[left],G,H)
             # = getting_best_split_paramters(X[left])[1]
             left_l,right_l = split(X[left],best_column_L,best_value_L)
             best_column_R,best_value_R,best_gain= getting_best_split_paramters(X[right],G,H)
             # = getting_best_split_paramters(X[right])[1]
             left_r,right_r = split(X[right],best_column_R,best_value_R)
        
       # Here i use argmax to know what is the majority class in every leaf
        
        class_left_l  = np.bincount(Y[left_l].astype(int)).argmax()
        class_left_r = np.bincount(Y[left_r].astype(int)).argmax()
        class_right_l = np.bincount(Y[right_l].astype(int)).argmax()
        class_right_r = np.bincount(Y[right_r].astype(int)).argmax()
        
        print("The indices of the left_l  leaf is ", left_l)
        print("-----------------------------------")
        print("The indices of the left_r  leaf is", left_r)
        print("----------------------------------")
        print("The indices of the right_l leaf is", right_l)
        print("------------------------------------")
        print("The indices of the right_r leaf is", right_r)
        print("------------------------------------")
        print("The majority class in left_l leaf is",class_left_l )
        print("------------------------------------")
        print(" The majority class in left_r lead is",class_left_r)
        print("------------------------------------")
        print(" The majority class in right_l lead is",class_right_l)
        print("------------------------------------")
        print(" The majority class in right_r lead is",class_right_r)
        
      # Here i calculate the error based on which original label is not equal to the majority class of the leaf
        error_left  = np.where(Y[left_l] != class_left_l) + np.where (Y[left_r] != class_left_r )
        error_right = np.where(Y[right_l] != class_right_l) + np.where(Y[right_r] != class_right_r)
        count_error_left  = len(error_left) 
        count_error_right = len(error_right)
                             
      # adding additive variable 
        additive_var[left_l]  = -1*(np.sum(G[left_l])/np.sum((H[left_l])+ lamb))                     
        additive_var[left_r]  =  -1*(np.sum(G[left_r])/np.sum((H[left_r])+ lamb))
        additive_var[right_l] =  -1*(np.sum(G[right_l])/np.sum((H[right_l])+ lamb))
        additive_var[right_r] =  -1*(np.sum(G[right_r])/np.sum((H[right_r])+ lamb))                   
      # the total error is equal to the error of the left and the right branch  
        total_error = count_error_left + count_error_right
        #print("the total error is", total_error)
        error[tree_val] =  total_error
        #print(sum(error.values()) / X.shape[0])        
    return (sum(error.values()) / X.shape[0])        
        



<b>gradient_boosting_trainng is to train</b>
1.the loss is 0.017



In [267]:
def gradient_boosting_training():
     loss = gradient_boosting(X,y,tree_num = 3 )
     print ("loss is", loss)    
gradient_boosting_training()        

1 0.8958247334604302 1.3289473684210527
this is the depthh 1
The indices of the left_l  leaf is  [  0   2   4   5   7  14  26  29  30  31  32  33  37  38  46  53  56  57
  58  63  65  66  72  75  78  83  87  88  89  91  93  95  97 101 102 103
 106 110 114 120 122 124 125 130 134 137 138 140 141 142 147 151 152 161
 169 175 177 185 186 190 197 200 202 205 206 207 214 215 218 223 225 231
 243 244 248 252 254 256 259 261 262 264 272 278 281 287 289 295 302 306
 307 312 315 320 321 330 333 334 337 342 344 346 347 348 352 356 358 359
 360 364 365 367 370 380 381 383 386 392 393 394 396 398 401 402 403 404
 409 415 422 424 425 431 437 439 440 442 447 450 451 452 453 455 456 457
 461 465 467 473 479 482 483 484 492 493 495 496 497 501 502 505 507 508
 509 512 514 515 520 523 526 527 528 530 534 537 542 547 552 553 555 557
 560 563 565 567 572 574 577 580 581 583 585 586 587 588 589 592 593 595
 598]
-----------------------------------
The indices of the left_r  leaf is [91]
------------------

In [270]:
def tunining_trees_num ( x = x_val, y= y_val):
    tree_space = [3, 5,7]
    error ={}
    for tree_num in tree_space :
       error[tree_num] = gradient_boosting(x,y,tree_num = tree_num )
       #print(error[tree_num])
    print (error) 
tunining_trees_num()   
    

0 1.8940262667990222 0.4107142857142858
this is the depthh 1
The indices of the left_l  leaf is  [  0   1   2   3   4   5   6   8   9  10  11  12  13  14  17  20  21  22
  23  24  25  27  28  30  31  32  33  34  35  37  38  39  40  42  43  48
  50  51  52  53  54  55  57  58  59  60  61  63  64  67  69  70  71  73
  74  75  76  79  81  82  83  85  87  88  90  91  93  96  97  98 100 101
 103 104 105 107 108 110 114 115 116 119 120 123 125 128 130 131 132 133
 134 138 139]
-----------------------------------
The indices of the left_r  leaf is [2]
----------------------------------
The indices of the right_l leaf is [  7  15  16  18  19  26  29  36  41  44  45  46  47  49  56  62  65  66
  68  72  77  78  80  84  86  89  92  94  95  99 102 106 109 111 112 113
 117 118 121 122 124 126 127 129 135 136 137]
------------------------------------
The indices of the right_r leaf is [0 1 3 4 5 6 7 8 9]
------------------------------------
The majority class in left_l leaf is 0
-------------------

<b>Runing tuning hyperparamter nuber of trees : </b>
    1. with 3 trees error = 0.08<br>
    2. with 5 trees errors = 0.133<br>
    3. with 7 trees errors = 0.1866<br>
the best number of trees is 3    

In [ ]:
def predict(X_test):
    